<a href="https://colab.research.google.com/github/Avinash10000/SARBnet/blob/main/SARBnet_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Step 2: Running The CNN on the Raspberry Pi4B

##Requirements

Run the list below and solve any dependency errors using sudo apt-get {module}

In [ ]:
absl-py==0.7.0
astor==0.7.1
automationhat==0.0.4
blinker==1.3
blinkt==0.1.1
Cap1xxx==0.1.3
chardet==2.3.0
click==6.6
colorama==0.3.7
cryptography==1.7.1
drumhat==0.1.0
envirophat==0.0.6
ExplorerHAT==0.4.2
Flask==0.12.1
fourletterphat==0.1.0
gast==0.2.2
gpiozero==1.4.1
grpcio==1.18.0
h5py==2.9.0
idna==2.2
itsdangerous==0.24
jedi==0.10.2
Jinja2==2.8
Keras==2.2.4
Keras-Applications==1.0.7
Keras-Preprocessing==1.0.9
keyring==10.1
keyrings.alt==1.3
Markdown==3.0.1
MarkupSafe==0.23
mcpi==0.1.1
microdotphat==0.1.3
mote==0.0.3
motephat==0.0.2
numpy==1.16.2
oauthlib==2.0.1
opencv-python==3.4.4.19
pgzero==1.2
phatbeat==0.1.0
pianohat==0.0.5
picamera==1.13
picraft==1.0
piglow==1.2.4
pigpio==1.38
Pillow==4.0.0
protobuf==3.6.1
pyasn1==0.1.9
pycrypto==2.6.1
pygame==1.9.3
pygobject==3.22.0
pyinotify==0.9.6
PyJWT==1.4.2
pyOpenSSL==16.2.0
pyserial==3.2.1
python-apt==1.1.0b5
pyxdg==0.25
PyYAML==3.13
rainbowhat==0.0.2
requests==2.12.4
requests-oauthlib==0.7.0
RPi.GPIO==0.6.3
RTIMULib==7.2.1
scipy==1.2.1
scrollphat==0.0.7
scrollphathd==1.2.0
SecretStorage==2.3.1
sense-emu==1.0
sense-hat==2.2.0
simplejson==3.10.0
six==1.12.0
skywriter==0.0.7
sn3218==1.2.7
spidev==3.3
tensorboard==1.11.0
tensorflow==1.11.0
termcolor==1.1.0
thonny==2.1.16
touchphat==0.0.1
twython==3.4.0
urllib3==1.19.1
virtualenv==16.4.0
Werkzeug==0.14.1

## Constants


These are the constant values needed for the whole code to function.

In [ ]:
#name the temporary image path
TEMP_IMG_PATH = "img.jpg"

#directory to store images that have been classified
STORE_DIRECTORY = "Classified_Images"


#input dimensions for the images
WIDTH = 64
HEIGHT = 64
CHANNELS=3


#these are the list of classes that we have 
CLASS_LIST = ['cardboard', 'compost', 'glass', 'metal', 'paper', 'plastic', 'trash']
''' need to change the next three'''
COMPOST_LIST = ['compost']
TRASH_LIST = ['trash']
CARDBOARD_LIST = ['cardboard']
GLASS_LIST = ['glass']
METAL_LIST = ['metal']
PAPER_LIST = ['paper']
PLASTIC_LIST = ['plastic']


## Taking the Picture **and** PiCamera/PIR Sensor/Servo Motor Code

In [ ]:
  
from time import sleep
import cv2
import picamera
import numpy as np
import io
from picamera.array import PiRGBArray
from picamera import PiCamera
from PIL import Image as Img
''' need to add PIR part'''
import RPi.GPIO as GPIO
import time
import picamera
import picamera.array
from picamera import PiCamera
camera = PiCamera()

GPIO.setmode(GPIO.BOARD)
GPIO.setwarnings(False)
GPIO.setup(11,GPIO.IN)
num=1

while GPIO.input(11)==0:
        time.sleep(0.5)
        
    else:
        
        #take the picture and convert it into an opencv object
	       camera.capture(stream, format="bgr")
	       image = stream.array
         cv2.imwrite(TEMP_IMG_PATH, image)
	       print(crop_img.shape)
	       camera.remove_overlay(o_sq)
	       sleep(1)
	       camera.stop_preview()
	       stream.truncate(0)
	       return image
                        
         while GPIO.input(11)==1:
           time.sleep(0.5)





## Doing the Image Classification on the Image



In [ ]:
''' waste-classifier'''

import os
import tensorflow as tf
import keras
from keras.models import load_model
import cv2
from glob import glob
import numpy as np
from constants import *


#load the pretrained model
model = load_model("model_clumped.h5py")


def scale_X(X):
    return X/255.0

#preprocess the image given to be classified
def process_single_img(image):
    # img = cv2.imread(TEMP_IMG_PATH)
    #resize and normalize images
    if CHANNELS==1:
        img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img, (WIDTH, HEIGHT))
    img = scale_X(img)
    img = img.reshape(WIDTH,HEIGHT,CHANNELS)
    return img

#predicts a single image given the numpy array of the image
def predict_single_img(img):
    #preprocesss the image
    processed_img = process_single_img(img)

    #predict the image using the preloaded model
    prediction = model.predict(np.array([processed_img]))
    pred = np.argmax(prediction)

    print(pred)
    #match the numerica predicted class to the name
    pred_class = CLASS_LIST[pred]
    print(pred_class)
    
    #sort into trash, recycling or compost
    waste_type = "trash"
    if pred_class in COMPOST_LIST:
	    waste_type = "compost"
    elif pred_class in METAL_LIST:
	    waste_type= "metal"
    elif pred_class in CARDBOARD_LIST:
      waste_type = 'cardboard'
    elif pred_class in GLASS_LIST:
      waste_type = 'glass'
    elif pred_class in PAPER_LIST:
      waste_type = 'paper'
    elif pred_class in PLASTIC_LIST:
      waste_type = 'plastic'
    return (waste_type , pred_class)



#store the specific given waste type in the appropriate folder with
#an enumerated name    
def store_in_folder(waste_type):
	parent_dir = STORE_DIRECTORY+"/"+waste_type+"/"
	num = len(glob(parent_dir+"*.jpg"))
	print("current num images:",num)
	os.rename(TEMP_IMG_PATH, parent_dir
		+waste_type +str(num+1)+".jpg")


# Import libraries
import RPi.GPIO as GPIO
import time


  
# Set GPIO numbering mode
GPIO.setmode(GPIO.BOARD)

# Set pin 12 as an output, and define as servo1 as PWM pin
GPIO.setup(12,GPIO.OUT)
servo1 = GPIO.PWM(12,50) # pin 11 for servo1, pulse 50Hz

# Start PWM running, with value of 0 (pulse off)
servo1.start(0)


def cleaning_things_at_the_end():
  #Clean things up at the end
    servo1.stop()
    GPIO.cleanup()
    print("Goodbye!")
  
# defining the movement (90 degrees clockwise)
# time statment can be anything
# ChangeDutyCycle = 0 removes jitters by servo motor
# adding 2 is necessary
def moving_item_to_left():
    servo1.ChangeDutyCycle(12)
    time.sleep(0.5)
    servo1.ChangeDutyCycle(0)
    time.sleep(3.5)
    servo1.ChangeDutyCycle(2)
    time.sleep(0.5)
    servo1.ChangeDutyCycle(0)
  
  
'''defining the movement (90 degrees anti-clockwise)'''
# time statment can be anything
# ChangeDutyCycle = 0 removes jitters by servo motor
# adding 2 is necessary
def moving_item_to_right():
    servo1.ChangeDutyCycle(12)
    time.sleep(0.5)
    servo1.ChangeDutyCycle(0)
    time.sleep(0.5)
    servo1.ChangeDutyCycle(2)
    time.sleep(0.5)
    servo1.ChangeDutyCycle(0)





predict_single_img(image)
if predicted_class == 'cardboard' or 'trash' or 'glass' or 'paper' or 'plastic':
  moving_item_to_left()
  cleaning_things_at_the_end()
elif predicted_class == 'compost' or 'trash':
  moving_item_to_right()
  cleaning_things_at_the_end()

    

    